In [16]:
from hgcal_dev.datasets.hcal import HCalDataModule
import numpy as np
import plotly
import plotly.express as px
import uproot
import pandas as pd
from tqdm import tqdm
from pathlib import Path
from plotly.subplots import make_subplots

In [17]:
root_parent_path = Path('/home/alexj/data/hgcal-dev/hcal_3')
raw_data_dir = Path('/home/alexj/data/hgcal-dev/hcal_3/raw')
data_dir = Path('/home/alexj/data/hgcal-dev/hcal_3/min_energy_0.0_min_hits_0')
transformed_data_dir = Path('/home/alexj/data/hgcal-dev/hcal_3/min_energy_0.5_min_hits_0')

In [18]:
root_dir = uproot.rootio.open(root_parent_path / '8Apr21_v2.root')

In [19]:
root_events = root_dir.get('Events;1')
df = pd.DataFrame()
for k, v in tqdm(root_events[b'RecHit'].items()):
    df[k.decode('ascii').split('.')[1]] = v.array()

for n in tqdm(range(df.shape[0])):
    jagged_event = df.loc[n]
    df_dict = {k: jagged_event[k] for k in jagged_event.keys()}
    flat_event = pd.DataFrame(df_dict)
    flat_event['hit'] = (flat_event['genE'] > 0.2).astype(int)
    flat_event.to_pickle(raw_data_dir / f'event_{n}.pkl')

 72%|███████▏  | 147187/204044 [05:16<02:02, 465.66it/s]


KeyboardInterrupt: 

In [13]:
HCalDataModule.merge_events(raw_data_dir, data_dir)

100%|██████████| 204044/204044 [18:23<00:00, 184.97it/s]


In [10]:
events = [f for f in transformed_data_dir.glob('*.pkl')][:1000]
raw_events = [f for f in data_dir.glob('*.pkl')][:1000]

In [6]:
evt =  pd.read_pickle(events[0])

In [15]:
evt.columns

Index(['fUniqueID', 'fBits', 'genE', 'energy', 'timeFalling', 'time', 'x', 'y',
       'z', 'eta', 'phi', 'RHClusterDist', 'hit', 'particleMatch', 'trackId',
       'nRHClusterMatches', 'RHClusterMatch', 'RHAntiKtCluster_gen',
       'distanceRHAntiKtCluster_gen', 'RHAntiKtCluster_reco',
       'distanceRHAntiKtCluster_reco', 'weta', 'wphi'],
      dtype='object')

In [12]:
raw_evt = pd.read_pickle(raw_events[0])
(raw_evt.hit==1).sum()

0